In [2]:
import sys

try:
    import src
except:
    sys.path.append('../')
    import src

import os
import matplotlib.pyplot as plt
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
from torch.utils.data import DataLoader
import torch
from src.dataset import WallADE20KDataset, SimpleWallADE20KDataset
from src import config
from src.model import WallModel
from torch.optim import Adam

In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [13]:
ENCODER_NAME = "mobileone_s4" # S4 has squeeze-excite blocks
# ENCODER_NAME = "mobileone_s4"
ARCHITECTURE_NAME = "DeepLabV3Plus"
# ARCHITECTURE_NAME = "DeepLabV3"
# ARCHITECTURE_NAME = "PSPNet"
# ARCHITECTURE_NAME = "PAN"
# ARCHITECTURE_NAME = "UNet"

model = WallModel(architecture=ARCHITECTURE_NAME, encoder_name=ENCODER_NAME, in_channels=3, out_classes=1, encoder_depth=5).eval()
# model = smp.create_model(
#     ARCHITECTURE_NAME,
#     encoder_name=ENCODER_NAME,
#     in_channels=3,
#     classes=1,
#     decoder_atrous_rates=(12, 24, 36),
#     decoder_channels=512,
#     we
# )
print(count_parameters(model))

rep_model = smp.encoders.mobileone.reparameterize_model(model).eval()
print(count_parameters(rep_model))

# mobileone_s4
# 16068729
# 3166481

# mobileone_s3
# 11285009
# 3163409

# mobileone_s2
# 8997521
# 3161873

# mobileone_s1
# 5702289
# 2158097

# mobileone_s0
# 6089473
# 1821201

16068729
3166481


In [10]:
x = torch.rand((1, 3, 704, 704)).cpu()

y = model.model.eval().encoder(x)

In [14]:
for o in y:
    print(o.shape)

torch.Size([1, 3, 704, 704])
torch.Size([1, 64, 352, 352])
torch.Size([1, 192, 176, 176])
torch.Size([1, 448, 88, 88])


In [26]:
x = torch.rand((8, 3, 512, 512))
y = model.eval()(x)
y[0].shape, y[1].shape

(torch.Size([8, 1, 512, 512]), torch.Size([8, 1]))

In [28]:
trainer = pl.Trainer(
    accelerator='cpu',
    max_epochs=5,
)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/ricardsku/miniconda3/envs/ml/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [29]:
model.optimizers()

RuntimeError: WallModel is not attached to a `Trainer`.

In [31]:
'3'.zfill(2)

'03'

In [24]:
dataset = WallADE20KDataset(root=config.DATA_ROOT, mode='all')
# dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=20)

In [25]:
image, mask = dataset[0]

In [26]:
type(image), image.shape, image.min(), image.max()

(numpy.ndarray, (1536, 2048, 3), 0, 255)

In [27]:
preprocess_fn = smp.encoders.get_preprocessing_fn(ENCODER_NAME, pretrained='imagenet')
preprocess_fn

functools.partial(<function preprocess_input at 0x7fb470af3e20>, input_space='RGB', input_range=[0, 1], mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [28]:
image_pre = preprocess_fn(image)
type(image_pre), image_pre.shape, image_pre.min(), image_pre.max()

(numpy.ndarray, (1536, 2048, 3), -2.1179039301310043, 2.6399999999999997)

In [32]:
simple_dataset = SimpleWallADE20KDataset(root=config.DATA_ROOT, mode='all')

In [34]:
image, mask = simple_dataset[0]
image = image.squeeze().transpose((1, 2, 0))
type(image), image.shape, image.min(), image.max()

(numpy.ndarray, (512, 512, 3), 1, 255)

In [35]:
params = smp.encoders.get_preprocessing_params(encoder_name=ENCODER_NAME)
params

{'input_space': 'RGB',
 'input_range': [0, 1],
 'mean': [0.485, 0.456, 0.406],
 'std': [0.229, 0.224, 0.225]}

In [36]:
dataloader = DataLoader(simple_dataset, batch_size=1, shuffle=True, num_workers=20)
image, mask = next(iter(dataloader))

In [37]:
std = torch.tensor(params["std"]).view(1, 3, 1, 1)
mean = torch.tensor(params["mean"]).view(1, 3, 1, 1)

In [38]:
image_pre = (image - mean) / std

In [2]:
repr(smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True))

'DiceLoss()'

In [ ]:
from segmentation_models_pytorch.datasets import SimpleOxfordPetDataset

In [ ]:
# Download data
root = "./dataset"
SimpleOxfordPetDataset.download(root)

In [ ]:
sample = train_dataset[0]

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, weight_decay=1e-4, momentum=0.9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(T_0=10, T_mult=2, optimizer=optimizer)

In [7]:
print(repr(optimizer))

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    initial_lr: 0.001
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0001
)


In [23]:
print(scheduler.state_dict())

{'T_0': 10, 'T_i': 10, 'T_mult': 2, 'eta_min': 0, 'T_cur': 0, 'base_lrs': [0.001], 'last_epoch': 0, 'verbose': False, '_step_count': 0, '_get_lr_called_within_step': False, '_last_lr': [0.001]}


In [21]:
print(type(scheduler).__name__)

CosineAnnealingWarmRestarts


In [24]:
scheduler.__dict__

{'T_0': 10,
 'T_i': 10,
 'T_mult': 2,
 'eta_min': 0,
 'T_cur': 0,
 'optimizer': SGD (
 Parameter Group 0
     dampening: 0
     differentiable: False
     foreach: None
     initial_lr: 0.001
     lr: 0.001
     maximize: False
     momentum: 0.9
     nesterov: False
     weight_decay: 0.0001
 ),
 'base_lrs': [0.001],
 'last_epoch': 0,
 'verbose': False,
 '_step_count': 0,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.001]}

In [9]:
print(type(optimizer).__name__)

SGD


In [12]:
print(optimizer.param_groups[0].keys())

dict_keys(['params', 'lr', 'momentum', 'dampening', 'weight_decay', 'nesterov', 'maximize', 'foreach', 'differentiable', 'initial_lr'])


In [18]:
optimizer.param_groups[0]['nesterov']

False

In [20]:
for (key, value) in optimizer.param_groups[0].items():
    if key == 'params':
        continue
    print(f'optimizer/{key}', value)

optimizer/lr 0.001
optimizer/momentum 0.9
optimizer/dampening 0
optimizer/weight_decay 0.0001
optimizer/nesterov False
optimizer/maximize False
optimizer/foreach None
optimizer/differentiable False
optimizer/initial_lr 0.001
